In [1]:
pip install PyMySQL

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import pymysql

# Establish connection to MySQL
myconnection = pymysql.connect(host='localhost', user='root', passwd='root', database='redbus_data')

# Read the CSV file
csv_file_path = r'C:\Users\kisho\Desktop\Janani\Capstone 1 red bus\redbus_data.csv'
df = pd.read_csv(csv_file_path)

# Print the columns to debug
print("Columns in DataFrame:", df.columns)

# Ensure correct column names and drop unnecessary columns
df = df.rename(columns=lambda x: x.strip())  # Remove leading/trailing whitespaces

# Rename columns to match expected names
df.rename(columns={
    'Bus name': 'Bus_Name',
    'Bus Type': 'Bus_Type',
    'Departure Time': 'Departure_Time',
    'Arrival Time': 'Arrival_Time',
    'Seat Availability': 'Seat_Availability',
    'Bus Link': 'Bus_Link',
    'Bus Route': 'Bus_Route'
}, inplace=True)

# Format the DataFrame columns
try:
    if 'Departure_Time' in df.columns:
        df['Departure_Time'] = pd.to_datetime(df['Departure_Time'], format='%H:%M', errors='coerce').dt.time
    if 'Arrival_Time' in df.columns:
        df['Arrival_Time'] = pd.to_datetime(df['Arrival_Time'], format='%H:%M', errors='coerce').dt.time
except Exception as e:
    print(f"Error parsing date columns: {e}")

# Clean 'Seat_Availability' column and handle NaN values
if 'Seat_Availability' in df.columns:
    df['Seat_Availability'] = df['Seat_Availability'].astype(str).str.extract(r'(\d+)').fillna(0).astype(int)
else:
    print("Warning: 'Seat_Availability' column not found")

# Ensure 'Rating' and 'Fare' columns are properly formatted
if 'Rating' in df.columns:
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(0)  # Convert to float and fill NaNs with 0
if 'Fare' in df.columns:
    df['Fare'] = pd.to_numeric(df['Fare'], errors='coerce').fillna(0)  # Convert to float and fill NaNs with 0

# Ensure 'Bus_Name' column is present and treated as string
if 'Bus_Name' in df.columns:
    df['Bus_Name'] = df['Bus_Name'].astype(str).fillna('Unknown')
else:
    print("Warning: 'Bus_Name' column not found")

# Map pandas dtypes to MySQL dtypes
dtype_mapping = {
    'object': 'TEXT',
    'int64': 'INT',
    'int32': 'INT',
    'float64': 'FLOAT',
    'datetime64[ns]': 'DATETIME',
    'datetime.time': 'TIME'
}

# Replace spaces with underscores in column names
df.columns = df.columns.str.replace(' ', '_')

# Generate the formatted string for creating the table with escaped column names and types
column_definitions = [f"`{col}` {dtype_mapping.get(str(df[col].dtype), 'TEXT')}" for col in df.columns]
table_definition = ", ".join(column_definitions)

table_name = "redbus"

# Create cursor object
cursor = myconnection.cursor()

# Drop table if it exists (optional)
cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

# Create table
create_table_query = f"CREATE TABLE {table_name} ({table_definition})"
print("Create Table Query:", create_table_query)  # Print the query to debug
cursor.execute(create_table_query)

# Insert data into the table
insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['%s'] * len(df.columns))})"

# Execute the insert queries
for row in df.itertuples(index=False, name=None):
    cursor.execute(insert_query, row)

# Commit the changes
myconnection.commit()

# Close the cursor and connection
cursor.close()
myconnection.close()

Columns in DataFrame: Index(['Route Name', 'Route Link', 'Bus Name', 'Bus Type', 'Departing Time',
       'Duration', 'Reaching Time', 'Star Rating', 'Price', 'Seats Available'],
      dtype='object')
Create Table Query: CREATE TABLE redbus (`Route_Name` TEXT, `Route_Link` TEXT, `Bus_Name` TEXT, `Bus_Type` TEXT, `Departing_Time` TEXT, `Duration` TEXT, `Reaching_Time` TEXT, `Star_Rating` TEXT, `Price` FLOAT, `Seats_Available` TEXT)


In [6]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seats_Available
0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 23700,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",10:15,05h 35m,15:50,4.0,436.0,32 Seats available
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3543,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",10:15,07h 05m,17:20,3.5\n18,469.0,27 Seats available
2,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 9310,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",10:15,06h 35m,16:50,4.5\n8,436.0,30 Seats available
3,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3657,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),10:30,05h 55m,16:25,3.9\n110,720.0,31 Seats available\n12 Window
4,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 3513,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",10:39,09h 06m,19:45,2.1\n6,469.0,32 Seats available
...,...,...,...,...,...,...,...,...,...,...
10457,Kolkata to Asansol (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...,Royal Cruiser,Scania Multi-Axle AC Semi Sleeper (2+2),19:30,04h 45m,00:15,3.7\n66,523.0,23 Seats available\n10 Window
10458,Kolkata to Asansol (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...,Beauty Travels,NON A/C Seater / Sleeper (2+2),20:20,04h 10m,00:30,3.0,380.0,36 Seats available\n18 Window
10459,Kolkata to Asansol (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...,Pariwartan Bus Service,NON A/C Seater / Sleeper (2+2),18:30,06h 00m,00:30,,500.0,54 Seats available\n26 Window
10460,Kolkata to Asansol (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...,Maharani Travels,A/C Seater / Sleeper (2+1),21:05,04h 55m,02:00,2.6,760.0,24 Seats available\n6 Single
